# Test consistency between `legacy` and new `output` module

*Test if the two `carsus` interfaces make identical atomic files.*

**IMPORTANT:** `pytest --nbval-lax` runs the notebook and evaluate cells marked with `#NBVAL_CHECK_OUTPUT` as tests by comparing the new cell output with the previously saved result, so think carefully before pushing changes.

The `assert_frame_equal` and `assert_series_equal` returns nothing when objects are identical.

---

In [1]:
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal

In [2]:
GFALL_IONS = "H-Be"
CHIANTI_IONS = "H-He"
CHIANTI_DB_VERSION = "9.0.1"

## 1. Create `old` atomic data

In [3]:
from carsus import init_db
from carsus.io.nist import NISTIonizationEnergiesIngester, NISTWeightsCompIngester
from carsus.io.kurucz import GFALLIngester
from carsus.io.chianti_ import ChiantiIngester
from carsus.io.zeta import KnoxLongZetaIngester
from carsus.io.output import AtomData

session = init_db()

weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()
session.commit()

ioniz_energies_ingester = NISTIonizationEnergiesIngester(session, spectra=GFALL_IONS)
ioniz_energies_ingester.ingest(ionization_energies=True, ground_levels=True)
session.commit()

gfall_ingester = GFALLIngester(session, ions=GFALL_IONS)
gfall_ingester.ingest(levels=True, lines=True)
session.commit()

chianti_ingester = ChiantiIngester(session, ions=CHIANTI_IONS)
chianti_ingester.ingest(levels=True, lines=True, collisions=True)
session.commit()

zeta_ingester = KnoxLongZetaIngester(session)
zeta_ingester.ingest()
session.commit()

old_atom_data = AtomData(session,
                         selected_atoms=GFALL_IONS, 
                         chianti_short_name=f"chianti_v{CHIANTI_DB_VERSION}", 
                         chianti_ions=CHIANTI_IONS)

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets
[                carsus.base][INFO] - Initializing the database at sqlite:// (base.py:32)
[                carsus.base][INFO] - Ingesting basic atomic data. (base.py:45)
[ carsus.io.nist.weightscomp][INFO] - Downloading data from the NIST Atomic Weights and Isotopic Compositions Database. (weightscomp.py:49)
[ carsus.io.nist.weightscomp][INFO] - Ingesting atomic weights from `nist`. (weightscomp.py:174)
[  carsus.io.nist.ionization][INFO] - Downloading ionization energies from the NIST Atomic Spectra Database. (ionization.py:65)
[  carsus.io.nist.ionization][INFO] - Ingesting ionization energies from `nist-asd`. (ionization.py:239)
[  carsus.io.nist.ionization][INFO] - Ingesting ground levels from `nist-asd`. (ionization.py:261)
[     carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:95)
[     carsus.io.kurucz

## 2. Create `new` atomic data

In [4]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData

atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies(GFALL_IONS)
gfall_reader = GFALLReader(ions=GFALL_IONS)
chianti_reader = ChiantiReader(ions=CHIANTI_IONS, collisions=True, priority=20)
zeta_data = KnoxLongZeta()

new_atom_data = TARDISAtomData(atomic_weights,
                               ionization_energies,
                               gfall_reader,
                               zeta_data,
                               chianti_reader)

[ carsus.io.nist.weightscomp][INFO] - Downloading data from the NIST Atomic Weights and Isotopic Compositions Database. (weightscomp.py:49)
[  carsus.io.nist.ionization][INFO] - Downloading ionization energies from the NIST Atomic Spectra Database. (ionization.py:65)
[     carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:95)
[      carsus.io.output.base][INFO] - Ingesting energy levels. (base.py:291)
[     carsus.io.kurucz.gfall][INFO] - Parsing GFALL from: https://media.githubusercontent.com/media/tardis-sn/carsus-db/master/gfall/gfall_latest.dat (gfall.py:148)
[      carsus.io.output.base][INFO] - GFALL selected species: Li 0, Li 1, Be 0, Be 1, Be 2. (base.py:328)
[      carsus.io.output.base][INFO] - Chianti selected species: H 0, He 0, He 1. (base.py:332)
[      carsus.io.output.base][INFO] - Ingesting transition lines. (base.py:384)
[     carsus.io.kurucz.gfall][INFO] - 

## 3. Create objects

In [5]:
# to do: make more consistent
old_atomic_weights = old_atom_data.atom_masses_prepared
old_atomic_weights['symbol'] = old_atomic_weights['symbol'].str.decode('UTF-8')
old_atomic_weights['name'] = old_atomic_weights['name'].str.decode('UTF-8')

new_atomic_weights = new_atom_data.atomic_weights.base.loc[1:4]  # H-Be

In [6]:
# to do: make more consistent
old_ionization_energies = old_atom_data.ionization_energies_prepared
new_ionization_energies = new_atom_data.ionization_energies.base

In [7]:
old_levels_all = old_atom_data._get_all_levels_data().reset_index()
new_levels_all = new_atom_data._get_all_levels_data().drop(columns=["ds_id"])

[      carsus.io.output.base][INFO] - Ingesting energy levels. (base.py:291)
[      carsus.io.output.base][INFO] - GFALL selected species: Li 0, Li 1, Be 0, Be 1, Be 2. (base.py:328)
[      carsus.io.output.base][INFO] - Chianti selected species: H 0, He 0, He 1. (base.py:332)


In [8]:
old_levels = old_atom_data.levels
new_levels = new_atom_data.levels.drop(columns=["ds_id"])

[                py.warnings][WARNING] - /home/epassaro/miniconda3/envs/carsus/lib/python3.6/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
 (warnings.py:99)


In [9]:
old_levels_prepared = old_atom_data.levels_prepared
new_levels_prepared = new_atom_data.levels_prepared

In [10]:
old_lines_all = old_atom_data._get_all_lines_data()
new_lines_all = new_atom_data._get_all_lines_data().drop(columns=["ds_id"])

[      carsus.io.output.base][INFO] - Ingesting transition lines. (base.py:384)
[      carsus.io.output.base][INFO] - Matching lines and levels. (base.py:423)


In [11]:
old_lines = old_atom_data.lines
new_lines = new_atom_data.lines.drop(columns=["ds_id"])

In [12]:
old_lines_prepared = old_atom_data.lines_prepared
new_lines_prepared = new_atom_data.lines_prepared

In [13]:
old_macro_atom = old_atom_data.macro_atom
new_macro_atom = new_atom_data.macro_atom

In [14]:
old_macro_atom_prepared = old_atom_data.macro_atom_prepared
new_macro_atom_prepared = new_atom_data.macro_atom_prepared

In [15]:
old_macro_atom_references = old_atom_data.macro_atom_references
new_macro_atom_references = new_atom_data.macro_atom_references

In [16]:
old_macro_atom_references_prepared = old_atom_data.macro_atom_references_prepared
new_macro_atom_references_prepared = new_atom_data.macro_atom_references_prepared

In [17]:
old_collisions = old_atom_data.collisions.drop(columns=["btemp", "bscups"])
new_collisions = new_atom_data.collisions.drop(columns=["btemp", "bscups"])

In [18]:
old_collisions_prepared = old_atom_data.collisions_prepared
new_collisions_prepared = new_atom_data.collisions_prepared

In [19]:
# to do: make more consistent
old_zeta_data = old_atom_data.zeta_data
new_zeta_data = new_atom_data.zeta_data.base

## 4. Assert equal

In [20]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_atomic_weights, new_atomic_weights)

In [21]:
#NBVAL_CHECK_OUTPUT
assert_series_equal(old_ionization_energies, new_ionization_energies)

In [22]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_levels_all, new_levels_all)

In [23]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_levels, new_levels)

In [24]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_levels_prepared, new_levels_prepared)

In [25]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_lines_all, new_lines_all)

In [26]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_lines, new_lines)

In [27]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_lines_prepared, new_lines_prepared)

In [28]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_macro_atom, new_macro_atom)

In [29]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_macro_atom_prepared, new_macro_atom_prepared)

In [30]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_macro_atom_references, new_macro_atom_references)

In [31]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_macro_atom_references_prepared, new_macro_atom_references_prepared)

In [32]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_collisions, new_collisions)

In [33]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_collisions_prepared, new_collisions_prepared)

In [34]:
#NBVAL_CHECK_OUTPUT
assert_frame_equal(old_zeta_data, new_zeta_data)